In [ ]:
# Author: Alireza P. Nouri
# Email: apashamoham@miners.utep.edu
# this is the code to implement different baseline models to classify pubmed articles

In [1]:
import torch
import random
import pickle
import warnings
from tqdm import tqdm
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [196]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
warnings.simplefilter("ignore", ResourceWarning)
warnings.filterwarnings('always') 

In [101]:
# generate bert embedding:
import sys
sys.path.append('..')
from dataset_reader import dataset_loader
from contextual_embedding.contextual_embedding import bert_embedding
from transformers import BertTokenizer

contextual_bert_dict= dict()
context_em = bert_embedding()

for id in tqdm(abstract_id_dict.keys()):
    temp_text = str(title_id_dict[id])+str(' ') +str(abstract_id_dict[id])
    contextual_bert_dict[id] =  context_em.embedding_generator(temp_text, embedding_type='summing')[1][0][1]


100%|██████████| 34973/34973 [52:18<00:00, 11.14it/s] 


In [102]:
pickle.dump(contextual_bert_dict, open("01_contextual_bert_dict.p", "wb"))

/tmp/ipykernel_1012541/1708544064.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='01_contextual_bert_dict.p'>
  pickle.dump(contextual_bert_dict, open("01_contextual_bert_dict.p", "wb"))


In [3]:

stories = pickle.load(open('01_stories.p','rb'))
title_id_dict = pickle.load(open('01_title_id_dict.p', 'rb'))
abstract_id_dict = pickle.load(open('01_abstract_id_dict.p', 'rb'))
text_id_dict = pickle.load(open('01_text_id_dict.p', 'rb'))
date_id_dict = pickle.load(open('01_date_id_dict.p', 'rb'))
meshMajor_id_dict = pickle.load(open('01_meshMajor_id_dict.p', 'rb'))
pmid_id_dict = pickle.load(open('01_pmid_id_dict.p', 'rb'))
meshid_id_dict = pickle.load(open('01_meshid_id_dict.p', 'rb'))
meshroot_id_dict = pickle.load(open('01_meshroot_id_dict.p', 'rb'))
train_id = pickle.load(open('01_train_id.p', 'rb'))
tf_idf_dict = pickle.load(open('01_tf_idf_dict.p', 'rb'))
contextual_doc_embedding = pickle.load(open('01_contextual_doc_embedding.p', 'rb'))
contextual_bert_embedding = pickle.load(open('01_contextual_bert_embedding.p', 'rb'))
contextual_dict = pickle.load(open('01_contextual_dict.p','rb'))
contextual_doc_embedding = pickle.load(open('01_contextual_doc_embedding.p', 'rb'))
dataset = pickle.load(open('01_dataset.p', 'rb'))
new_stories = pickle.load(open('01_final_stories.p', 'rb'))

/tmp/ipykernel_1012541/2909961077.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='01_stories.p'>
  stories = pickle.load(open('01_stories.p','rb'))
/tmp/ipykernel_1012541/2909961077.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='01_title_id_dict.p'>
  title_id_dict = pickle.load(open('01_title_id_dict.p', 'rb'))
/tmp/ipykernel_1012541/2909961077.py:3: ResourceWarning: unclosed file <_io.BufferedReader name='01_abstract_id_dict.p'>
  abstract_id_dict = pickle.load(open('01_abstract_id_dict.p', 'rb'))
/tmp/ipykernel_1012541/2909961077.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='01_text_id_dict.p'>
  text_id_dict = pickle.load(open('01_text_id_dict.p', 'rb'))
/tmp/ipykernel_1012541/2909961077.py:5: ResourceWarning: unclosed file <_io.BufferedReader name='01_date_id_dict.p'>
  date_id_dict = pickle.load(open('01_date_id_dict.p', 'rb'))
/tmp/ipykernel_1012541/2909961077.py:6: ResourceWarning: unclosed file <_io.BufferedReader name='01_meshMa

In [332]:
new_stories_1 = pickle.load(open('02_final_stories.p', 'rb'))

/tmp/ipykernel_1012541/331937566.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='01_final_stories.p'>
  new_stories_1 = pickle.load(open('01_final_stories.p', 'rb'))


In [5]:
# doc_id: all ids in stories generated by difstorygen
# total_id: all ids in dataset that has timestamp
# test_id: all ids in dataset that has timestamp but they are not in stories generated by difstorygen
# total_dataset: a list of data in dataset with values of id, title, abstract, and list of meshterms 
# total_train: all data in doc_id with values of id, title, abstract, and list of meshterms
# total_test: all data in test_id with values of id, title, abstract, and list of meshterms
# stories_neighbors: a dictionary with all neighbors of documents generated by difstorygen
# training_ids: a list of ids of data with neighbors generated by difstorygen 
# testing_ids: a list of ids of data from total_id that not in training_ids


# tf_idf_neighbors: dictionary of tf-idf of enighbors of document in training_ids
# contextual_neighbors: a dictionary of contextual embedding of documents in training_ids


# print("doc_id length is {}".format(len(doc_id))) 
# print("total_id length is {}".format(len(total_id))) 
# print("test_id length is {}".format(len(test_id)))
# print("total_dataset length is {}".format(len(total_dataset))) 
# print("total_train length is {}".format(len(total_train))) 
# print("total_test length is {}".format(len(total_test)))
# print("stories_neighbors length is {}".format(len(stories_neighbors))) 
# print("training_ids length is {}".format(len(training_ids))) 
# print("testing_ids length is {}".format(len(testing_ids))) 
# print("tf_idf_neighbors length is {}".format(len(tf_idf_neighbors))) 
# print("contextual_neighbors length is {}".format(len(contextual_neighbors))) 

In [5]:
# new_stories = pickle.load(open('01_final_stories.p', 'rb'))

difStoryGen_stories = dict()
for i in new_stories.keys():
    difStoryGen_stories[i] = new_stories[i]
for i in stories.keys():
    difStoryGen_stories[i] = stories[i]
print('Size of difStoryGen_stories dict is {}'.format(len(difStoryGen_stories)))  

Size of difStoryGen_stories dict is 300


In [335]:
for i in new_stories_1.keys():
    difStoryGen_stories[i] = new_stories_1[i]

In [336]:
print('Size of difStoryGen_stories dict is {}'.format(len(difStoryGen_stories)))  

Size of difStoryGen_stories dict is 300


In [58]:
label_dict= dict()
for d in dataset:
    label_dict[d['id']] = list(d.values())[7:-1]
label_size = len(label_dict[list(label_dict.keys())[0]]) 
print('lable size is {}'.format(label_size))

lable size is 14


In [6]:
difstoryGen_ids = set()
for i in difStoryGen_stories.keys():
    for j in difStoryGen_stories[i]:
        difstoryGen_ids.add(j)
print('Size of difstoryGen_ids ids is {}'.format(len(difstoryGen_ids)))  

Size of difstoryGen_ids ids is 2572


In [7]:
total_dataset_ids = set()
for i in title_id_dict:
    total_dataset_ids.add(i)
print('Size of total_dataset_ids ids is {}'.format(len(total_dataset_ids)))  

Size of total_dataset_ids ids is 34973


In [8]:
not_difstoryGen_ids = set()
for i in total_dataset_ids:
    if i not in difstoryGen_ids:
        not_difstoryGen_ids.add(i)
print('Size of not_difstoryGen_ids ids is {}'.format(len(not_difstoryGen_ids)))        

Size of not_difstoryGen_ids ids is 32401


In [9]:
n_sampling_for_converter = int(len(difstoryGen_ids)/4)
testing_for_converter_ids = random.sample(list(not_difstoryGen_ids), n_sampling_for_converter)
print('Size of training dataset for convertor is {}'.format(len(testing_for_converter_ids)))

Size of training dataset for convertor is 643


In [10]:
contextual_ids = set()
for i in contextual_dict.keys():
    contextual_ids.add(i)
print('Size of contextual embedding ids is {}'.format(len(contextual_ids)))

Size of contextual embedding ids is 27544


In [11]:
tf_idf_ids = set()
for i in tf_idf_dict.keys():
    tf_idf_ids.add(i)
print('Size of tf_idf_ids ids is {}'.format(len(tf_idf_ids)))

Size of tf_idf_ids ids is 34973


In [12]:
#generate the neighbors
stories_neighbors = dict()
for i in difStoryGen_stories:
    for j in difStoryGen_stories[i]:
        stories_neighbors[j] = []

for i in list(difStoryGen_stories.keys())[1:-1]:
    if len(difStoryGen_stories[i]) < 2:
        continue
    stories_neighbors[difStoryGen_stories[i][0]].append(difStoryGen_stories[i][1])
    for j in range(1,len(difStoryGen_stories[i])-1):
        stories_neighbors[difStoryGen_stories[i][j]].append(difStoryGen_stories[i][j+1])
        stories_neighbors[difStoryGen_stories[i][j]].append(difStoryGen_stories[i][j-1])
    stories_neighbors[difStoryGen_stories[i][-1]].append(difStoryGen_stories[i][-2])
print('Size of stories_neighbors ids is {}'.format(len(stories_neighbors)))

Size of stories_neighbors ids is 2572


In [107]:
training_for_convertor_ids  = [i for i in stories_neighbors.keys() if len(stories_neighbors[i])>0]
# #generate tf-idf of neighbors:
# tf_idf_neighbors = dict()
# for i in tqdm(training_ids):
#     tf_idf_neighbors[i] = np.average([tf_idf_dict[j] for j in stories_neighbors[i]], axis=0)

#generate contextual embedding of neighbors
contextual_neighbors = dict()
for i in tqdm(training_for_convertor_ids):
    contextual_neighbors[i] = np.average([contextual_dict[j] for j in stories_neighbors[i]], axis=0)
print('Size of contextual_neighbors dict is {}'.format(len(contextual_neighbors)))
print('Size of training_for_convertor_ids list is {}'.format(len(training_for_convertor_ids)))


contextual_neighbors_768 = dict()
for i in tqdm(training_for_convertor_ids):
    contextual_neighbors_768[i] = np.average([contextual_bert_dict[j] for j in stories_neighbors[i]], axis=0)
print('Size of contextual_neighbors dict is {}'.format(len(contextual_neighbors_768)))
print('Size of training_for_convertor_ids list is {}'.format(len(training_for_convertor_ids)))

100%|██████████| 2544/2544 [00:00<00:00, 63253.95it/s]


Size of contextual_neighbors dict is 2544
Size of training_for_convertor_ids list is 2544


100%|██████████| 2544/2544 [00:00<00:00, 57452.20it/s]

Size of contextual_neighbors dict is 2544
Size of training_for_convertor_ids list is 2544


In [25]:
# a neural network to predict the neighbors for any given contextual embedding
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
X = np.array([contextual_dict[i] for i in training_for_convertor_ids]) #training data for the neural network to generate the neighbors embedding vector
y = np.array([contextual_neighbors[i] for i in training_for_convertor_ids]) #training data for the neural network to map the neighbors embedding vector
# Define the model
def mlp_model(input_dim, output_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),  # First hidden layer with 64 neurons and ReLU activation
        BatchNormalization(),
        Dropout(0.25),
        Dense(128, activation='relu'),  # Second hidden layer with 32 neurons and ReLU activation
        BatchNormalization(),
        Dropout(0.25),
        Dense(64, activation='relu'), 
        Dense(output_dim, activation='linear')  # Output layer with 'output_dim' neurons
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Example usage
input_dim = X.shape[1] # Size of the input vector (n)
output_dim = y.shape[1]  # Size of the output vector (m)

# Create the MLP model
contextual_model = mlp_model(input_dim, output_dim)

# Model summary to see the architecture
contextual_model.summary()


# Train the model
contextual_model.fit(X, y, epochs=300, batch_size=16)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 batch_normalization_2 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 128)               8320      
                                                                 
 batch_normalization_3 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 128)              

In [26]:
Z = np.array([contextual_dict[i] for i in not_difstoryGen_ids if i in contextual_ids])

generated_neighbors = contextual_model.predict(Z)

Z_ids = [i for i in not_difstoryGen_ids if i in contextual_ids]

generated_contexual_dict = dict()
for i,j in zip(Z_ids, generated_neighbors):
    generated_contexual_dict[i] = j

781/781 [==============================] - 1s 624us/step


In [109]:
# a neural network to predict the neighbors for any given contextual embedding
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
X = np.array([contextual_neighbors_768[i] for i in training_for_convertor_ids]) #training data for the neural network to generate the neighbors embedding vector
y = np.array([contextual_neighbors_768[i] for i in training_for_convertor_ids]) #training data for the neural network to map the neighbors embedding vector
# Define the model
def mlp_model(input_dim, output_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),  # First hidden layer with 64 neurons and ReLU activation
        BatchNormalization(),
        Dropout(0.25),
        Dense(128, activation='relu'),  # Second hidden layer with 32 neurons and ReLU activation
        BatchNormalization(),
        Dropout(0.25),
        Dense(64, activation='relu'), 
        Dense(output_dim, activation='linear')  # Output layer with 'output_dim' neurons
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Example usage
input_dim = X.shape[1] # Size of the input vector (n)
output_dim = y.shape[1]  # Size of the output vector (m)

# Create the MLP model
contextual_model_768 = mlp_model(input_dim, output_dim)

# Model summary to see the architecture
contextual_model_768.summary()


# Train the model
contextual_model_768.fit(X, y, epochs=300, batch_size=16)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                49216     
                                                                 
 batch_normalization_6 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 128)               8320      
                                                                 
 batch_normalization_7 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dropout_7 (Dropout)         (None, 128)              

In [140]:
Z_768 = np.array([contextual_neighbors_768[i] for i in contextual_neighbors_768.keys()])

generated_neighbors_768 = contextual_model_768.predict(Z_768)

Z_ids_768 = [i for i in not_difstoryGen_ids if i in contextual_bert_dict.keys()]

generated_contexual_dict_768 = dict()
for i,j in zip(Z_ids, generated_neighbors_768):
    generated_contexual_dict_768[i] = j

80/80 [==============================] - 0s 690us/step


In [31]:
print('Size of generated_contexual_dict dict is {}'.format(len(generated_contexual_dict)))

Size of generated_contexual_dict dict is 24972


In [33]:
l_total = len(generated_contexual_dict)
ids_for_training_basedlines = list(generated_contexual_dict.keys())[int(l_total/5):]
ids_for_test_basedlines = list(generated_contexual_dict.keys())[:int(l_total/5)]
print('training size is {}'.format(len(ids_for_training_basedlines)))
print('test size is {}'.format(len(ids_for_test_basedlines)))

In [144]:
X_train_orig = np.array([contextual_dict[i] for i in ids_for_training_basedlines])
X_test_orig = np.array([contextual_dict[i] for i in ids_for_test_basedlines])
X_train_orig_768 = np.array([contextual_bert_dict[i] for i in ids_for_training_basedlines])
X_test_orig_768 = np.array([contextual_bert_dict[i] for i in ids_for_test_basedlines])
X_train_generated = np.array([np.array(list(generated_contexual_dict[i])+list(contextual_dict[i])) for i in ids_for_training_basedlines])
X_test_generated = np.array([np.array(list(generated_contexual_dict[i])+ list(contextual_dict[i])) for i in ids_for_test_basedlines])
X_train_generated_768 = np.array([np.array(list(generated_contexual_dict_768[i])+list(contextual_bert_dict[i])) for i in list(generated_contexual_dict_768.keys())[:2000]])
X_test_generated_768 = np.array([np.array(list(generated_contexual_dict_768[i])+ list(contextual_bert_dict[i])) for i in list(generated_contexual_dict_768.keys())[2000:]])

In [142]:
len(generated_contexual_dict_768.keys())

2544

In [147]:
# classification based on MultinomialNB
accuracy_reslts_MultinomialNB_c = []
precision_results_MultinomialNB_c = []
recall_results_MultinomialNB_c = []
f1_results_MultinomialNB_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_orig = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
    # model = MultinomialNB()
    p_orig.fit(X_train_orig, y_train)
    
    y_pred = p_orig.predict(X_test_orig)
    accuracy_reslts_MultinomialNB_c.append(accuracy_score(y_test, y_pred))
    precision_results_MultinomialNB_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_MultinomialNB_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_MultinomialNB_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for naive bayes, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_MultinomialNB_c))
print("Average of percision: ", np.average(precision_results_MultinomialNB_c))
print("Average of recall: ", np.average(recall_results_MultinomialNB_c))
print("Average of f1 score: ", np.average(f1_results_MultinomialNB_c))

100%|██████████| 14/14 [00:00<00:00, 26.37it/s]

results for naive bayes, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.7637164597517022
Average of percision:  0.7059762154437905
Average of recall:  0.7637164597517022
Average of f1 score:  0.6886155262473939


In [148]:
# classification based on MultinomialNB

accuracy_reslts_MultinomialNB_g = []
precision_results_MultinomialNB_g = []
recall_results_MultinomialNB_g = []
f1_results_MultinomialNB_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
    # model = MultinomialNB()
    
    p.fit(X_train_generated , y_train)
    
    y_pred = p.predict(X_test_generated)
    accuracy_reslts_MultinomialNB_g.append(accuracy_score(y_test, y_pred))
    precision_results_MultinomialNB_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_MultinomialNB_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_MultinomialNB_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for naive bayes, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_MultinomialNB_g))
print("Average of percision: ", np.average(precision_results_MultinomialNB_g))
print("Average of recall: ", np.average(recall_results_MultinomialNB_g))
print("Average of f1 score: ", np.average(f1_results_MultinomialNB_g))

100%|██████████| 14/14 [00:00<00:00, 26.67it/s]

results for naive bayes, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.7663195835002004
Average of percision:  0.7054111290455791
Average of recall:  0.7663195835002004
Average of f1 score:  0.6942685660820348


In [149]:
# classification based on MultinomialNB
accuracy_reslts_MultinomialNB_c768 = []
precision_results_MultinomialNB_c768 = []
recall_results_MultinomialNB_c768 = []
f1_results_MultinomialNB_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_orig = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
    # model = MultinomialNB()
    p_orig.fit(X_train_generated_768, y_train)
    
    y_pred = p_orig.predict(X_test_generated_768)
    accuracy_reslts_MultinomialNB_c768.append(accuracy_score(y_test, y_pred))
    precision_results_MultinomialNB_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_MultinomialNB_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_MultinomialNB_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for naive bayes, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_MultinomialNB_c768))
print("Average of percision: ", np.average(precision_results_MultinomialNB_c768))
print("Average of recall: ", np.average(recall_results_MultinomialNB_c768))
print("Average of f1 score: ", np.average(f1_results_MultinomialNB_c768))

100%|██████████| 14/14 [00:00<00:00, 52.49it/s]

results for naive bayes, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.8048844537815124
Average of percision:  0.7716651512509944
Average of recall:  0.8048844537815124
Average of f1 score:  0.7808605133866585


In [331]:
#SVM classification without difstorygen based on contextual bert document embedding

accuracy_reslts_svm_c = []
precision_results_svm_c = []
recall_results_svm_c = []
f1_results_svm_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines] # list of categories based on MeSH terms
    
    model_svm_c = svm.SVC()
    model_svm_c.fit(X_train_orig, y_train)
    
    y_pred = model_svm_c.predict(X_test_orig)
    accuracy_reslts_svm_c.append(accuracy_score(y_test, y_pred))
    precision_results_svm_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_svm_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_svm_c.append(f1_score(y_test, y_pred, average='weighted'))
import numpy as np
print("results for svm, using contextual bert embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_svm_c))
print("Average of percision: ", np.average(precision_results_svm_c))
print("Average of recall: ", np.average(recall_results_svm_c))
print("Average of f1 score: ", np.average(f1_results_svm_c))

100%|██████████| 14/14 [01:38<00:00,  7.06s/it]

results for svm, using contextual bert embedding vectors without difstoryGen
Average  of accuracy:  0.8394215916242349
Average of percision:  0.8330968351287306
Average of recall:  0.8394215916242349
Average of f1 score:  0.8157445198055508


In [154]:
#SVM classification with difstorygen based on contextual bert (64) document embedding

accuracy_reslts_svm_g = []
precision_results_svm_g = []
recall_results_svm_g = []
f1_results_svm_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_svm = Pipeline([('Normalizing',MinMaxScaler()),('SVM',svm.SVC())])
    
    
    p_svm.fit(X_train_generated , y_train)
    
    y_pred = p_svm.predict(X_test_generated)
    accuracy_reslts_svm_g.append(accuracy_score(y_test, y_pred))
    precision_results_svm_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_svm_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_svm_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for svm, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_svm_g))
print("Average of percision: ", np.average(precision_results_svm_g))
print("Average of recall: ", np.average(recall_results_svm_g))
print("Average of f1 score: ", np.average(f1_results_svm_g))

100%|██████████| 14/14 [02:09<00:00,  9.25s/it]

results for svm, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.8323988786543853
Average of percision:  0.787869736613021
Average of recall:  0.8323988786543853
Average of f1 score:  0.8040846439358439


In [155]:
#SVM classification with difstorygen based on contextual bert (768) document embedding

accuracy_reslts_svm_c768 = []
precision_results_svm_c768 = []
recall_results_svm_c768 = []
f1_results_svm_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_svm_c768 = Pipeline([('Normalizing',MinMaxScaler()),('SVM',svm.SVC())])
    
    p_svm_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_svm_c768.predict(X_test_generated_768)
    
    accuracy_reslts_svm_c768.append(accuracy_score(y_test, y_pred))
    precision_results_svm_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_svm_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_svm_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for svm, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_svm_c768))
print("Average of percision: ", np.average(precision_results_svm_c768))
print("Average of recall: ", np.average(recall_results_svm_c768))
print("Average of f1 score: ", np.average(f1_results_svm_c768))

100%|██████████| 14/14 [00:07<00:00,  1.90it/s]

results for svm, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.8405987394957984
Average of percision:  0.7976229726526894
Average of recall:  0.8405987394957984
Average of f1 score:  0.8119404303993726


In [169]:
# classification based on KNeighborsClassifier
accuracy_reslts_KNeighborsClassifier_c = []
precision_results_KNeighborsClassifier_c = []
recall_results_KNeighborsClassifier_c = []
f1_results_KNeighborsClassifier_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_KNeighborsClassifier= Pipeline([('Normalizing',MinMaxScaler()),('KNeighborsClassifier',KNeighborsClassifier(n_neighbors=3))])
    
    p_KNeighborsClassifier.fit(X_train_orig, y_train)
    
    y_pred = p_KNeighborsClassifier.predict(X_test_orig)
    accuracy_reslts_KNeighborsClassifier_c.append(accuracy_score(y_test, y_pred))
    precision_results_KNeighborsClassifier_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_KNeighborsClassifier_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_KNeighborsClassifier_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for KNeighborsClassifier, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_KNeighborsClassifier_c))
print("Average of percision: ", np.average(precision_results_KNeighborsClassifier_c))
print("Average of recall: ", np.average(recall_results_KNeighborsClassifier_c))
print("Average of f1 score: ", np.average(f1_results_KNeighborsClassifier_c))


100%|██████████| 14/14 [00:00<00:00, 14.74it/s]

results for KNeighborsClassifier, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.7956118771096745
Average of percision:  0.7814267691717725
Average of recall:  0.7956118771096745
Average of f1 score:  0.7858529274219761


In [170]:
# classification based on KNeighborsClassifier for bert 64 with difstorygen

accuracy_reslts_KNeighborsClassifier_g = []
precision_results_KNeighborsClassifier_g = []
recall_results_KNeighborsClassifier_g = []
f1_results_KNeighborsClassifier_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_KNeighborsClassifier_g = Pipeline([('Normalizing',MinMaxScaler()),('KNeighborsClassifier',KNeighborsClassifier(n_neighbors=3))])
    
    
    p_KNeighborsClassifier_g.fit(X_train_generated , y_train)
    
    y_pred = p_KNeighborsClassifier_g.predict(X_test_generated)
    accuracy_reslts_KNeighborsClassifier_g.append(accuracy_score(y_test, y_pred))
    precision_results_KNeighborsClassifier_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_KNeighborsClassifier_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_KNeighborsClassifier_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for KNeighborsClassifier, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_KNeighborsClassifier_g))
print("Average of percision: ", np.average(precision_results_KNeighborsClassifier_g))
print("Average of recall: ", np.average(recall_results_KNeighborsClassifier_g))
print("Average of f1 score: ", np.average(f1_results_KNeighborsClassifier_g))

100%|██████████| 14/14 [00:01<00:00, 10.84it/s]

results for KNeighborsClassifier, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.7957263001315864
Average of percision:  0.7818894279998526
Average of recall:  0.7957263001315864
Average of f1 score:  0.7861866197656456


In [179]:
# classification based on KNeighborsClassifier for bert 768 with difstorygen
accuracy_reslts_KNeighborsClassifier_c768 = []
precision_results_KNeighborsClassifier_c768 = []
recall_results_KNeighborsClassifier_c768 = []
f1_results_KNeighborsClassifier_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_KNeighborsClassifier_c768 = Pipeline([('Normalizing',MinMaxScaler()),('KNeighborsClassifier',KNeighborsClassifier(n_neighbors=3))])

    
    p_KNeighborsClassifier_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_KNeighborsClassifier_c768.predict(X_test_generated_768)
    
    accuracy_reslts_KNeighborsClassifier_c768.append(accuracy_score(y_test, y_pred))
    precision_results_KNeighborsClassifier_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_KNeighborsClassifier_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_KNeighborsClassifier_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for KNeighborsClassifier, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_KNeighborsClassifier_c768))
print("Average of percision: ", np.average(precision_results_KNeighborsClassifier_c768))
print("Average of recall: ", np.average(recall_results_KNeighborsClassifier_c768))
print("Average of f1 score: ", np.average(f1_results_KNeighborsClassifier_c768))

100%|██████████| 14/14 [00:00<00:00, 15.75it/s]

results for KNeighborsClassifier, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.7922794117647057
Average of percision:  0.7753753203349091
Average of recall:  0.7922794117647057
Average of f1 score:  0.7792481901446606


In [188]:
# classification based on Decision tree without difstorygen
accuracy_reslts_DecisionTreeClassifier_c = []
precision_results_DecisionTreeClassifier_c = []
recall_results_DecisionTreeClassifier_c = []
f1_results_DecisionTreeClassifier_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_DecisionTreeClassifier = Pipeline([('Normalizing',MinMaxScaler()),('DecisionTreeClassifier',DecisionTreeClassifier(criterion="entropy", max_depth=3))])
    p_DecisionTreeClassifier.fit(X_train_orig, y_train)
    
    y_pred = p_DecisionTreeClassifier.predict(X_test_orig)
    accuracy_reslts_DecisionTreeClassifier_c.append(accuracy_score(y_test, y_pred))
    precision_results_DecisionTreeClassifier_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_DecisionTreeClassifier_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_DecisionTreeClassifier_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for DecisionTreeClassifier, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_DecisionTreeClassifier_c))
print("Average of percision: ", np.average(precision_results_DecisionTreeClassifier_c))
print("Average of recall: ", np.average(recall_results_DecisionTreeClassifier_c))
print("Average of f1 score: ", np.average(f1_results_DecisionTreeClassifier_c))

100%|██████████| 14/14 [00:04<00:00,  3.13it/s]

results for DecisionTreeClassifier, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.776117054751416
Average of percision:  0.7135251536007265
Average of recall:  0.776117054751416
Average of f1 score:  0.737243041425166


In [186]:
# classification based on Decision tree with difstorygen embedding 64

accuracy_reslts_DecisionTreeClassifier_g = []
precision_results_DecisionTreeClassifier_g = []
recall_results_DecisionTreeClassifier_g = []
f1_results_DecisionTreeClassifier_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_DecisionTreeClassifier_g = Pipeline([('Normalizing',MinMaxScaler()),('DecisionTreeClassifier',DecisionTreeClassifier(criterion="entropy", max_depth=3))])
    
    p_DecisionTreeClassifier_g.fit(X_train_generated , y_train)
    
    y_pred = p_DecisionTreeClassifier_g.predict(X_test_generated)
    accuracy_reslts_DecisionTreeClassifier_g.append(accuracy_score(y_test, y_pred))
    precision_results_DecisionTreeClassifier_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_DecisionTreeClassifier_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_DecisionTreeClassifier_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for DecisionTreeClassifier, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_DecisionTreeClassifier_g))
print("Average of percision: ", np.average(precision_results_DecisionTreeClassifier_g))
print("Average of recall: ", np.average(recall_results_DecisionTreeClassifier_g))
print("Average of f1 score: ", np.average(f1_results_DecisionTreeClassifier_g))

100%|██████████| 14/14 [00:11<00:00,  1.26it/s]

results for DecisionTreeClassifier, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.775974025974026
Average of percision:  0.6977231942028509
Average of recall:  0.775974025974026
Average of f1 score:  0.7275317000623819


In [187]:
# classification based on Decision tree with difstorygen embedding 768
accuracy_reslts_DecisionTreeClassifier_c768 = []
precision_results_DecisionTreeClassifier_c768 = []
recall_results_DecisionTreeClassifier_c768 = []
f1_results_DecisionTreeClassifier_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_orig = Pipeline([('Normalizing',MinMaxScaler()),('DecisionTreeClassifier',DecisionTreeClassifier(criterion="entropy", max_depth=3))])
    p_orig.fit(X_train_generated_768, y_train)
    
    y_pred = p_orig.predict(X_test_generated_768)
    accuracy_reslts_DecisionTreeClassifier_c768.append(accuracy_score(y_test, y_pred))
    precision_results_DecisionTreeClassifier_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_DecisionTreeClassifier_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_DecisionTreeClassifier_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for DecisionTreeClassifier, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_DecisionTreeClassifier_c768))
print("Average of percision: ", np.average(precision_results_DecisionTreeClassifier_c768))
print("Average of recall: ", np.average(recall_results_DecisionTreeClassifier_c768))
print("Average of f1 score: ", np.average(f1_results_DecisionTreeClassifier_c768))



100%|██████████| 14/14 [00:07<00:00,  1.97it/s]

results for DecisionTreeClassifier, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.7851890756302521
Average of percision:  0.7685538295023456
Average of recall:  0.7851890756302521
Average of f1 score:  0.7624755372459633


In [194]:
# classification based on LogisticRegression()
accuracy_reslts_LogisticRegression_c = []
precision_results_LogisticRegression_c = []
recall_results_LogisticRegression_c = []
f1_results_LogisticRegression_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_LogisticRegression = Pipeline([('Normalizing',MinMaxScaler()),('LogisticRegression',LogisticRegression(solver='lbfgs', max_iter=1000))])
    p_LogisticRegression.fit(X_train_orig, y_train)
    
    y_pred = p_LogisticRegression.predict(X_test_orig)
    accuracy_reslts_LogisticRegression_c.append(accuracy_score(y_test, y_pred))
    precision_results_LogisticRegression_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_LogisticRegression_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_LogisticRegression_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for LogisticRegression, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_LogisticRegression_c))
print("Average of percision: ", np.average(precision_results_LogisticRegression_c))
print("Average of recall: ", np.average(recall_results_LogisticRegression_c))
print("Average of f1 score: ", np.average(f1_results_LogisticRegression_c))

100%|██████████| 14/14 [00:01<00:00,  7.61it/s]

results for LogisticRegression, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.8338577721837634
Average of percision:  0.8199932312861806
Average of recall:  0.8338577721837634
Average of f1 score:  0.8172561161753802


In [193]:
# classification based on LogisticRegression for bert 64

accuracy_reslts_LogisticRegression_g = []
precision_results_LogisticRegression_g = []
recall_results_LogisticRegression_g = []
f1_results_LogisticRegression_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_LogisticRegression_g = Pipeline([('Normalizing',MinMaxScaler()),('LogisticRegression',LogisticRegression(solver='lbfgs', max_iter=1000))])
    
    p_LogisticRegression_g.fit(X_train_generated , y_train)
    
    y_pred = p_LogisticRegression_g.predict(X_test_generated)
    accuracy_reslts_LogisticRegression_g.append(accuracy_score(y_test, y_pred))
    precision_results_LogisticRegression_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_LogisticRegression_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_LogisticRegression_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for LogisticRegression, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_LogisticRegression_g))
print("Average of percision: ", np.average(precision_results_LogisticRegression_g))
print("Average of recall: ", np.average(recall_results_LogisticRegression_g))
print("Average of f1 score: ", np.average(f1_results_LogisticRegression_g))

100%|██████████| 14/14 [00:07<00:00,  1.92it/s]

results for LogisticRegression, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.8343583729046283
Average of percision:  0.8212390285304422
Average of recall:  0.8343583729046283
Average of f1 score:  0.8177816168521


In [195]:
# classification based on LogisticRegression for bert 768
accuracy_reslts_LogisticRegression_c768 = []
precision_results_LogisticRegression_c768 = []
recall_results_LogisticRegression_c768 = []
f1_results_LogisticRegression_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_LogisticRegression_c768 = Pipeline([('Normalizing',MinMaxScaler()),('LogisticRegression',LogisticRegression(solver='lbfgs', max_iter=1000))])
    p_LogisticRegression_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_LogisticRegression_c768.predict(X_test_generated_768)
    accuracy_reslts_LogisticRegression_c768.append(accuracy_score(y_test, y_pred))
    precision_results_LogisticRegression_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_LogisticRegression_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_LogisticRegression_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for LogisticRegression_c768, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_LogisticRegression_c768))
print("Average of percision: ", np.average(precision_results_LogisticRegression_c768))
print("Average of recall: ", np.average(recall_results_LogisticRegression_c768))
print("Average of f1 score: ", np.average(f1_results_LogisticRegression_c768))

100%|██████████| 14/14 [00:02<00:00,  6.84it/s]

results for LogisticRegression_c768, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.8432247899159664
Average of percision:  0.8342457710971877
Average of recall:  0.8432247899159664
Average of f1 score:  0.8369003770788215


In [197]:
# classification based on GaussianNB
accuracy_reslts_GaussianNB_c = []
precision_results_GaussianNB_c = []
recall_results_GaussianNB_c = []
f1_results_GaussianNB_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_GaussianNB = Pipeline([('Normalizing',MinMaxScaler()),('GaussianNB',GaussianNB())])
    p_GaussianNB.fit(X_train_orig, y_train)
    
    y_pred = p_GaussianNB.predict(X_test_orig)
    accuracy_reslts_GaussianNB_c.append(accuracy_score(y_test, y_pred))
    precision_results_GaussianNB_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_GaussianNB_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_GaussianNB_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for p_GaussianNB, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_GaussianNB_c))
print("Average of percision: ", np.average(precision_results_GaussianNB_c))
print("Average of recall: ", np.average(recall_results_GaussianNB_c))
print("Average of f1 score: ", np.average(f1_results_GaussianNB_c))

100%|██████████| 14/14 [00:00<00:00, 34.96it/s]

results for p_GaussianNB, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.7916499799759712
Average of percision:  0.795747832689037
Average of recall:  0.7916499799759712
Average of f1 score:  0.7929831585908256


In [198]:
# classification based on GaussianNB with difstorygen bert 64

accuracy_reslts_GaussianNB_g = []
precision_results_GaussianNB_g = []
recall_results_GaussianNB_g = []
f1_results_GaussianNB_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_GaussianNB_g = Pipeline([('Normalizing',MinMaxScaler()),('GaussianNB',GaussianNB())])
    
    p_GaussianNB_g.fit(X_train_generated , y_train)
    
    y_pred = p_GaussianNB_g.predict(X_test_generated)
    accuracy_reslts_GaussianNB_g.append(accuracy_score(y_test, y_pred))
    precision_results_GaussianNB_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_GaussianNB_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_GaussianNB_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for GaussianNB, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_GaussianNB_g))
print("Average of percision: ", np.average(precision_results_GaussianNB_g))
print("Average of recall: ", np.average(recall_results_GaussianNB_g))
print("Average of f1 score: ", np.average(f1_results_GaussianNB_g))

100%|██████████| 14/14 [00:00<00:00, 32.50it/s]

results for GaussianNB, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.697179472509869
Average of percision:  0.7570874998417961
Average of recall:  0.697179472509869
Average of f1 score:  0.7010706211056047


In [211]:
# classification based on GaussianNB with difstorygen bert 768
accuracy_reslts_GaussianNB_c768 = []
precision_results_GaussianNB_c768 = []
recall_results_GaussianNB_c768 = []
f1_results_GaussianNB_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_GaussianNB_c768 = Pipeline([('Normalizing',MinMaxScaler()),('GaussianNB',GaussianNB())])
    p_GaussianNB_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_GaussianNB_c768.predict(X_test_generated_768)
    accuracy_reslts_GaussianNB_c768.append(accuracy_score(y_test, y_pred))
    precision_results_GaussianNB_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_GaussianNB_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_GaussianNB_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for GaussianNB, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_GaussianNB_c768))
print("Average of percision: ", np.average(precision_results_GaussianNB_c768))
print("Average of recall: ", np.average(recall_results_GaussianNB_c768))
print("Average of f1 score: ", np.average(f1_results_GaussianNB_c768))

100%|██████████| 14/14 [00:00<00:00, 55.21it/s]

results for GaussianNB, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.7322741596638656
Average of percision:  0.7990654883234248
Average of recall:  0.7322741596638656
Average of f1 score:  0.753926579547728


In [212]:
from sklearn.linear_model import SGDClassifier

In [247]:
# classification based on SGDClassifier
accuracy_reslts_SGDClassifier_c = []
precision_results_SGDClassifier_c = []
recall_results_SGDClassifier_c = []
f1_results_SGDClassifier_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_SGDClassifier = Pipeline([('Normalizing',MinMaxScaler()),('SGDClassifier',SGDClassifier(learning_rate='optimal', max_iter=1000, tol=1e-3))])
    p_SGDClassifier.fit(X_train_orig, y_train)
    
    y_pred = p_SGDClassifier.predict(X_test_orig)
    accuracy_reslts_SGDClassifier_c.append(accuracy_score(y_test, y_pred))
    precision_results_SGDClassifier_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_SGDClassifier_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_SGDClassifier_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for SGDClassifier, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_SGDClassifier_c))
print("Average of percision: ", np.average(precision_results_SGDClassifier_c))
print("Average of recall: ", np.average(recall_results_SGDClassifier_c))
print("Average of f1 score: ", np.average(f1_results_SGDClassifier_c))

100%|██████████| 14/14 [00:01<00:00, 11.19it/s]

results for SGDClassifier, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.8288660678528521
Average of percision:  0.7898654865956052
Average of recall:  0.8288660678528521
Average of f1 score:  0.8018011659427879


In [221]:
# classification based on SGDClassifier for bert 64 with difstorygen

accuracy_reslts_SGDClassifier_g = []
precision_results_SGDClassifier_g = []
recall_results_SGDClassifier_g = []
f1_results_SGDClassifier_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_SGDClassifier_g = Pipeline([('Normalizing',MinMaxScaler()),('SGDClassifier',SGDClassifier(learning_rate='optimal', max_iter=1000, tol=1e-3))])
    
    p_SGDClassifier_g.fit(X_train_generated , y_train)
    
    y_pred = p_SGDClassifier_g.predict(X_test_generated)
    accuracy_reslts_SGDClassifier_g.append(accuracy_score(y_test, y_pred))
    precision_results_SGDClassifier_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_SGDClassifier_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_SGDClassifier_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for SGDClassifier, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_SGDClassifier_g))
print("Average of percision: ", np.average(precision_results_SGDClassifier_g))
print("Average of recall: ", np.average(recall_results_SGDClassifier_g))
print("Average of f1 score: ", np.average(f1_results_SGDClassifier_g))

100%|██████████| 14/14 [00:03<00:00,  4.25it/s]

results for SGDClassifier, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.8276646261227759
Average of percision:  0.8071243099843535
Average of recall:  0.8276646261227759
Average of f1 score:  0.8038232711716501


In [248]:
# classification based on SGDClassifier for bert 768 with difstorygen
accuracy_reslts_SGDClassifier_c768 = []
precision_results_SGDClassifier_c768 = []
recall_results_SGDClassifier_c768 = []
f1_results_SGDClassifier_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_SGDClassifier_c768 = Pipeline([('Normalizing',MinMaxScaler()),('SGDClassifier',SGDClassifier(learning_rate='optimal',max_iter=1000))])
    p_SGDClassifier_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_SGDClassifier_c768.predict(X_test_generated_768)
    accuracy_reslts_SGDClassifier_c768.append(accuracy_score(y_test, y_pred))
    precision_results_SGDClassifier_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_SGDClassifier_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_SGDClassifier_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for SGDClassifier, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_SGDClassifier_c768))
print("Average of percision: ", np.average(precision_results_SGDClassifier_c768))
print("Average of recall: ", np.average(recall_results_SGDClassifier_c768))
print("Average of f1 score: ", np.average(f1_results_SGDClassifier_c768))

100%|██████████| 14/14 [00:03<00:00,  4.12it/s]

results for naive bayes, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.8233981092436976
Average of percision:  0.8243457521034975
Average of recall:  0.8233981092436976
Average of f1 score:  0.8147744971219907


In [262]:
# classification based on RandomForestClassifier
accuracy_reslts_RandomForestClassifier_c = []
precision_results_RandomForestClassifier_c = []
recall_results_RandomForestClassifier_c = []
f1_results_RandomForestClassifier_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_RandomForestClassifier = Pipeline([('Normalizing',MinMaxScaler()),('RandomForestClassifier',RandomForestClassifier(n_estimators =100))])
    p_RandomForestClassifier.fit(X_train_orig, y_train)
    
    y_pred = p_RandomForestClassifier.predict(X_test_orig)
    accuracy_reslts_RandomForestClassifier_c.append(accuracy_score(y_test, y_pred))
    precision_results_RandomForestClassifier_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_RandomForestClassifier_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_RandomForestClassifier_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for RandomForestClassifier, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_RandomForestClassifier_c))
print("Average of percision: ", np.average(precision_results_RandomForestClassifier_c))
print("Average of recall: ", np.average(recall_results_RandomForestClassifier_c))
print("Average of f1 score: ", np.average(f1_results_RandomForestClassifier_c))

100%|██████████| 14/14 [02:00<00:00,  8.62s/it]

results for RandomForestClassifier, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.8219577779049144
Average of percision:  0.8113334876281909
Average of recall:  0.8219577779049144
Average of f1 score:  0.7942972923930105


In [261]:
# classification based on RandomForestClassifier with difstorygen with bert 64

accuracy_reslts_RandomForestClassifier_g = []
precision_results_RandomForestClassifier_g = []
recall_results_RandomForestClassifier_g = []
f1_results_RandomForestClassifier_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_RandomForestClassifier_g = Pipeline([('Normalizing',MinMaxScaler()),('RandomForestClassifier',RandomForestClassifier(n_estimators =100))])
    
    p_RandomForestClassifier_g.fit(X_train_generated , y_train)
    
    y_pred = p_RandomForestClassifier_g.predict(X_test_generated)
    accuracy_reslts_RandomForestClassifier_g.append(accuracy_score(y_test, y_pred))
    precision_results_RandomForestClassifier_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_RandomForestClassifier_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_RandomForestClassifier_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for RandomForestClassifier, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_RandomForestClassifier_g))
print("Average of percision: ", np.average(precision_results_RandomForestClassifier_g))
print("Average of recall: ", np.average(recall_results_RandomForestClassifier_g))
print("Average of f1 score: ", np.average(f1_results_RandomForestClassifier_g))

100%|██████████| 14/14 [03:40<00:00, 15.72s/it]

results for RandomForestClassifier, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.816451169975399
Average of percision:  0.8055700964901785
Average of recall:  0.816451169975399
Average of f1 score:  0.7870176422299624


In [257]:
# classification based on RandomForestClassifier with difstorygen with bert 768
accuracy_reslts_RandomForestClassifier_c768 = []
precision_results_RandomForestClassifier_c768 = []
recall_results_RandomForestClassifier_c768 = []
f1_results_RandomForestClassifier_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_RandomForestClassifier_c768 = Pipeline([('Normalizing',MinMaxScaler()),('RandomForestClassifier',RandomForestClassifier(n_estimators =100))])
    p_RandomForestClassifier_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_RandomForestClassifier_c768.predict(X_test_generated_768)
    accuracy_reslts_RandomForestClassifier_c768.append(accuracy_score(y_test, y_pred))
    precision_results_RandomForestClassifier_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_RandomForestClassifier_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_RandomForestClassifier_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for RandomForestClassifier, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_RandomForestClassifier_c768))
print("Average of percision: ", np.average(precision_results_RandomForestClassifier_c768))
print("Average of recall: ", np.average(recall_results_RandomForestClassifier_c768))
print("Average of f1 score: ", np.average(f1_results_RandomForestClassifier_c768))


100%|██████████| 14/14 [00:38<00:00,  2.73s/it]

results for RandomForestClassifier, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.8247111344537815
Average of percision:  0.8116804740587625
Average of recall:  0.8247111344537815
Average of f1 score:  0.7983613254288996


In [263]:
# classification based on GradientBoostingClassifier
accuracy_reslts_GradientBoostingClassifier_c = []
precision_results_GradientBoostingClassifier_c = []
recall_results_GradientBoostingClassifier_c = []
f1_results_GradientBoostingClassifier_c = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    
    p_GradientBoostingClassifier = Pipeline([('Normalizing',MinMaxScaler()),('GradientBoostingClassifier',GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1))])
    p_GradientBoostingClassifier.fit(X_train_orig, y_train)
    
    y_pred = p_GradientBoostingClassifier.predict(X_test_orig)
    accuracy_reslts_GradientBoostingClassifier_c.append(accuracy_score(y_test, y_pred))
    precision_results_GradientBoostingClassifier_c.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_GradientBoostingClassifier_c.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_GradientBoostingClassifier_c.append(f1_score(y_test, y_pred, average='weighted'))

print("results for GradientBoostingClassifier, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_GradientBoostingClassifier_c))
print("Average of percision: ", np.average(precision_results_GradientBoostingClassifier_c))
print("Average of recall: ", np.average(recall_results_GradientBoostingClassifier_c))
print("Average of f1 score: ", np.average(f1_results_GradientBoostingClassifier_c))

100%|██████████| 14/14 [01:43<00:00,  7.41s/it]

results for GradientBoostingClassifier, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.8244607815092396
Average of percision:  0.8100160003168922
Average of recall:  0.8244607815092396
Average of f1 score:  0.8101287018228306


In [264]:
# classification based on GradientBoostingClassifier with difstoryGen and bert 64

accuracy_reslts_GradientBoostingClassifier_g = []
precision_results_GradientBoostingClassifier_g = []
recall_results_GradientBoostingClassifier_g = []
f1_results_GradientBoostingClassifier_g = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in ids_for_training_basedlines]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in ids_for_test_basedlines]  # list of categories based on MeSH terms
    p_GradientBoostingClassifier_g = Pipeline([('Normalizing',MinMaxScaler()),('GradientBoostingClassifier',GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1))])
    
    p_GradientBoostingClassifier_g.fit(X_train_generated , y_train)
    
    y_pred = p_GradientBoostingClassifier_g.predict(X_test_generated)
    accuracy_reslts_GradientBoostingClassifier_g.append(accuracy_score(y_test, y_pred))
    precision_results_GradientBoostingClassifier_g.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_GradientBoostingClassifier_g.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_GradientBoostingClassifier_g.append(f1_score(y_test, y_pred, average='weighted'))

print("results for GradientBoostingClassifier, using contextual embedding 64 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_GradientBoostingClassifier_g))
print("Average of percision: ", np.average(precision_results_GradientBoostingClassifier_g))
print("Average of recall: ", np.average(recall_results_GradientBoostingClassifier_g))
print("Average of f1 score: ", np.average(f1_results_GradientBoostingClassifier_g))

100%|██████████| 14/14 [04:25<00:00, 18.95s/it]

results for GradientBoostingClassifier, using contextual embedding 64 vectors with difstoryGen
Average  of accuracy:  0.8239887865438525
Average of percision:  0.8091933284376933
Average of recall:  0.8239887865438525
Average of f1 score:  0.809696693437858


In [267]:
# classification based on GradientBoostingClassifier with difstoryGen and bert 768

accuracy_reslts_GradientBoostingClassifier_c768 = []
precision_results_GradientBoostingClassifier_c768 = []
recall_results_GradientBoostingClassifier_c768 = []
f1_results_GradientBoostingClassifier_c768 = []
for r in tqdm(range(label_size)):
    # Example training data
    y_train = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[:2000]]  # list of categories based on MeSH terms
    y_test = [label_dict[i][r] for i in list(generated_contexual_dict_768.keys())[2000:]]  # list of categories based on MeSH terms
    
    p_GradientBoostingClassifier_c768 = Pipeline([('Normalizing',MinMaxScaler()),('GradientBoostingClassifier',GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1))])
    p_GradientBoostingClassifier_c768.fit(X_train_generated_768, y_train)
    
    y_pred = p_GradientBoostingClassifier_c768.predict(X_test_generated_768)
    accuracy_reslts_GradientBoostingClassifier_c768.append(accuracy_score(y_test, y_pred))
    precision_results_GradientBoostingClassifier_c768.append(precision_score(y_test, y_pred, zero_division=0, average='weighted'))
    recall_results_GradientBoostingClassifier_c768.append(recall_score(y_test, y_pred, average='weighted'))
    f1_results_GradientBoostingClassifier_c768.append(f1_score(y_test, y_pred, average='weighted'))

print("results for GradientBoostingClassifier, using bert 768 vectors with difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts_GradientBoostingClassifier_c768))
print("Average of percision: ", np.average(precision_results_GradientBoostingClassifier_c768))
print("Average of recall: ", np.average(recall_results_GradientBoostingClassifier_c768))
print("Average of f1 score: ", np.average(f1_results_GradientBoostingClassifier_c768))


100%|██████████| 14/14 [02:47<00:00, 11.97s/it]

results for GradientBoostingClassifier, using bert 768 vectors with difstoryGen
Average  of accuracy:  0.7912289915966387
Average of percision:  0.7832770882505814
Average of recall:  0.7912289915966387
Average of f1 score:  0.7867078084546495


In [184]:
#classify by RNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np


X_train = [contextual_dict[i[0]] for i in total_train if i[0] in id_contextual]  # list of abstracts
X_test = [contextual_dict[i[0]] for i in total_test if i[0] in id_contextual] 
# Assuming 'document_embeddings' is a list of embeddings and 'labels' is your list of labels
document_embeddings = [contextual_dict[i[0]] for i in total_train if i[0] in id_contextual]

accuracy_reslts = []
precision_results = []
recall_results = []
f1_results = []
for r in tqdm(range(len(total_train[0])-3)):
    # Example training data
    labels = [i[r+3] for i in total_train if i[0] in id_contextual ]  # list of categories based on MeSH terms
    y_test = [i[r+3] for i in total_test if i[0] in id_contextual]  # list of categories based on MeSH terms
    # Encode labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    encoded_labels = tf.keras.utils.to_categorical(encoded_labels)

    padded_docs = pad_sequences(document_embeddings, padding='post')
    model = Sequential([
        # Assuming embedding dimension is 100. Adjust 'input_dim' to your vocabulary size,
        # 'output_dim' to the size of your embeddings, and 'input_length' to the length of your padded documents
        Embedding(input_dim=len(X_train[0]), output_dim=2, input_length=padded_docs.shape[1]),
        SimpleRNN(units=64),
        Dense(units=len(np.unique(labels)), activation='softmax')
    ])
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(padded_docs, encoded_labels, epochs=10, verbose=1)

    new_padded_docs = pad_sequences(X_test, maxlen=padded_docs.shape[1], padding='post')
    # Predict
    predictions = model.predict(new_padded_docs)
    # Decode predicted labels
    predicted_labels = label_encoder.inverse_transform([np.argmax(pred) for pred in predictions])

    accuracy_reslts.append(accuracy_score(y_test, predicted_labels))
    precision_results.append(precision_score(y_test, predicted_labels, zero_division=0, average='weighted'))
    recall_results.append(recall_score(y_test, predicted_labels, average='weighted'))
    f1_results.append(f1_score(y_test, predicted_labels, average='weighted'))

print("results for RNN, using contextual embedding vectors without difstoryGen")
print("Average  of accuracy: ", np.average(accuracy_reslts))
print("Average of percision: ", np.average(precision_results))
print("Average of recall: ", np.average(recall_results))
print("Average of f1 score: ", np.average(f1_results))

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch 1/10
45/45 [==============================] - 2s 35ms/step - loss: 0.6944 - accuracy: 0.5092
Epoch 2/10
45/45 [==============================] - 1s 22ms/step - loss: 0.6878 - accuracy: 0.5452
Epoch 3/10
45/45 [==============================] - 1s 22ms/step - loss: 0.6823 - accuracy: 0.5840
Epoch 4/10
45/45 [==============================] - 1s 26ms/step - loss: 0.6822 - accuracy: 0.5614
Epoch 5/10
45/45 [==============================] - 1s 29ms/step - loss: 0.6613 - accuracy: 0.6109
Epoch 6/10
45/45 [==============================] - 1s 28ms/step - loss: 0.6756 - accuracy: 0.5939
Epoch 7/10
45/45 [==============================] - 1s 29ms/step - loss: 0.6575 - accuracy: 0.6349
Epoch 8/10
45/45 [==============================] - 1s 27ms/step - loss: 0.6610 - accuracy: 0.6158
Epoch 9/10
45/45 [==============================] - 1s 27ms/step - loss: 0.6799 - accuracy: 0.5946
Epoch 10/10
8/8 [==============================] - 0s 4ms/step


  7%|▋         | 1/14 [00:13<02:51, 13.15s/it]

Epoch 1/10
45/45 [==============================] - 2s 37ms/step - loss: 0.2226 - accuracy: 0.9449
Epoch 2/10
45/45 [==============================] - 1s 25ms/step - loss: 0.1801 - accuracy: 0.9576
Epoch 3/10
45/45 [==============================] - 1s 21ms/step - loss: 0.1809 - accuracy: 0.9576
Epoch 4/10
45/45 [==============================] - 1s 21ms/step - loss: 0.1762 - accuracy: 0.9576
Epoch 5/10
45/45 [==============================] - 1s 20ms/step - loss: 0.1762 - accuracy: 0.9576
Epoch 6/10
45/45 [==============================] - 1s 20ms/step - loss: 0.1781 - accuracy: 0.9576
Epoch 7/10
45/45 [==============================] - 1s 24ms/step - loss: 0.1749 - accuracy: 0.9576
Epoch 8/10
45/45 [==============================] - 1s 23ms/step - loss: 0.1741 - accuracy: 0.9576
Epoch 9/10
45/45 [==============================] - 1s 24ms/step - loss: 0.1725 - accuracy: 0.9576
Epoch 10/10
8/8 [==============================] - 0s 3ms/step


 14%|█▍        | 2/14 [00:24<02:27, 12.26s/it]

...

Epoch 1/10
45/45 [==============================] - 2s 30ms/step - loss: 0.6768 - accuracy: 0.5643
Epoch 2/10
45/45 [==============================] - 1s 21ms/step - loss: 0.6372 - accuracy: 0.6398
Epoch 3/10
45/45 [==============================] - 1s 22ms/step - loss: 0.5953 - accuracy: 0.6949
Epoch 4/10
45/45 [==============================] - 1s 20ms/step - loss: 0.5614 - accuracy: 0.7225
Epoch 5/10
45/45 [==============================] - 1s 21ms/step - loss: 0.5299 - accuracy: 0.7493
Epoch 6/10
45/45 [==============================] - 1s 20ms/step - loss: 0.5363 - accuracy: 0.7451
Epoch 7/10
45/45 [==============================] - 1s 19ms/step - loss: 0.5113 - accuracy: 0.7648
Epoch 8/10
45/45 [==============================] - 1s 19ms/step - loss: 0.5055 - accuracy: 0.7620
Epoch 9/10
45/45 [==============================] - 1s 21ms/step - loss: 0.5128 - accuracy: 0.7514
Epoch 10/10
8/8 [==============================] - 0s 4ms/step


 93%|█████████▎| 13/14 [02:33<00:11, 11.23s/it]

Epoch 1/10
45/45 [==============================] - 2s 32ms/step - loss: 0.4480 - accuracy: 0.8397
Epoch 2/10
45/45 [==============================] - 1s 22ms/step - loss: 0.4232 - accuracy: 0.8538
Epoch 3/10
45/45 [==============================] - 1s 33ms/step - loss: 0.4195 - accuracy: 0.8538
Epoch 4/10
45/45 [==============================] - 1s 20ms/step - loss: 0.4136 - accuracy: 0.8538
Epoch 5/10
45/45 [==============================] - 1s 22ms/step - loss: 0.4140 - accuracy: 0.8538
Epoch 6/10
45/45 [==============================] - 1s 24ms/step - loss: 0.4090 - accuracy: 0.8538
Epoch 7/10
45/45 [==============================] - 1s 19ms/step - loss: 0.4016 - accuracy: 0.8538
Epoch 8/10
45/45 [==============================] - 1s 19ms/step - loss: 0.3850 - accuracy: 0.8552
Epoch 9/10
45/45 [==============================] - 1s 22ms/step - loss: 0.3691 - accuracy: 0.8538
Epoch 10/10
8/8 [==============================] - 0s 4ms/step


100%|██████████| 14/14 [02:44<00:00, 11.78s/it]

results for RNN, using contextual embedding vectors without difstoryGen
Average  of accuracy:  0.7869147659063626
Average of percision:  0.7261407473902883
Average of recall:  0.7869147659063626
Average of f1 score:  0.7446993735893557


In [187]:
# #classify by RNN
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from sklearn.preprocessing import LabelEncoder
# import numpy as np


# X_train = [tf_idf_dict[i[0]] for i in total_train]  # list of abstracts
# X_test = [tf_idf_dict[i[0]] for i in total_test] 
# # Assuming 'document_embeddings' is a list of embeddings and 'labels' is your list of labels
# document_embeddings = [tf_idf_dict[i[0]] for i in total_train]

# accuracy_reslts = []
# precision_results = []
# recall_results = []
# f1_results = []
# for r in tqdm(range(len(total_train[0])-3)):
#     # Example training data
#     labels = [i[r+3] for i in total_train if i[0] in id_contextual ]  # list of categories based on MeSH terms
#     y_test = [i[r+3] for i in total_test if i[0] in id_contextual]  # list of categories based on MeSH terms
#     # Encode labels
#     label_encoder = LabelEncoder()
#     encoded_labels = label_encoder.fit_transform(labels)
#     encoded_labels = tf.keras.utils.to_categorical(encoded_labels)

#     padded_docs = pad_sequences(document_embeddings, padding='post')
#     model = Sequential([
#         # Assuming embedding dimension is 100. Adjust 'input_dim' to your vocabulary size,
#         # 'output_dim' to the size of your embeddings, and 'input_length' to the length of your padded documents
#         Embedding(input_dim=len(X_train[0]), output_dim=2, input_length=padded_docs.shape[1]),
#         SimpleRNN(units=64),
#         Dense(units=len(np.unique(labels)), activation='softmax')
#     ])
#     # Compile the model
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#     # Train the model
#     model.fit(padded_docs, encoded_labels, epochs=10, verbose=1)

#     new_padded_docs = pad_sequences(X_test, maxlen=padded_docs.shape[1], padding='post')
#     # Predict
#     predictions = model.predict(new_padded_docs)
#     # Decode predicted labels
#     predicted_labels = label_encoder.inverse_transform([np.argmax(pred) for pred in predictions])

#     accuracy_reslts.append(accuracy_score(y_test, predicted_labels))
#     precision_results.append(precision_score(y_test, predicted_labels, zero_division=0, average='weighted'))
#     recall_results.append(recall_score(y_test, predicted_labels, average='weighted'))
#     f1_results.append(f1_score(y_test, predicted_labels, average='weighted'))

# print("results for RNN, using tf-idf vectors without difstoryGen")
# print("Average  of accuracy: ", np.average(accuracy_reslts))
# print("Average of percision: ", np.average(precision_results))
# print("Average of recall: ", np.average(recall_results))
# print("Average of f1 score: ", np.average(f1_results))

100%|██████████| 1402/1402 [00:00<00:00, 64527.05it/s]


In [294]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Define the model
def mlp_model(input_dim, output_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),  # First hidden layer with 64 neurons and ReLU activation
        BatchNormalization(),
        Dense(128, activation='relu'),  # Second hidden layer with 32 neurons and ReLU activation
        BatchNormalization(),
        Dense(64, activation='relu'), 
        Dense(output_dim, activation='linear')  # Output layer with 'output_dim' neurons
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Prepare your input data (X) and labels (y)
X = np.array([contextual_dict[i] for i in training_ids])
y = np.array([contextual_neighbors[i] for i in training_ids])
# Example usage
input_dim = X.shape[1] # Size of the input vector (n)
output_dim = y.shape[1]  # Size of the output vector (m)

# Create the MLP model
contextual_model = mlp_model(input_dim, output_dim)

# Model summary to see the architecture
contextual_model.summary()


# X = np.random.rand(number_of_samples, input_dim)
# y = np.random.randint(output_dim, size=(number_of_samples, output_dim))
# y = tf.keras.utils.to_categorical(y, num_classes=output_dim)

# Train the model
contextual_model.fit(X, y, epochs=100, batch_size=16)

# Note: In this example, 'X' should be a NumPy array or a TensorFlow tensor with shape (number_of_samples, input_dim),
# where 'number_of_samples' is the total number of input vectors you have, and 'y' should be the corresponding labels
# for each input vector, one-hot encoded to have a shape of (number_of_samples, output_dim).


Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_75 (Dense)            (None, 64)                4160      
                                                                 
 batch_normalization_20 (Ba  (None, 64)                256       
 tchNormalization)                                               
                                                                 
 dense_76 (Dense)            (None, 128)               8320      
                                                                 
 batch_normalization_21 (Ba  (None, 128)               512       
 tchNormalization)                                               
                                                                 
 dense_77 (Dense)            (None, 64)                8256      
                                                                 
 dense_78 (Dense)            (None, 64)              

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Define the model
def mlp_model(input_dim, output_dim):
    model = Sequential([
        Dense(int(input_dim/10), activation='relu', input_shape=(input_dim,)),  # First hidden layer with 64 neurons and ReLU activation
        BatchNormalization(),
        Dropout(0.5),
        # Dense(int(input_dim/10), activation='relu'),  # Second hidden layer with 32 neurons and ReLU activation
        # BatchNormalization(),
        # Dropout(0.5),
        # Dense(output_dim, activation='relu'), 
        Dense(output_dim, activation='linear')  # Output layer with 'output_dim' neurons
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Prepare your input data (X) and labels (y)
X = np.array([tf_idf_dict[i] for i in training_ids])
y = np.array([tf_idf_neighbors[i] for i in training_ids])
# Example usage
input_dim = X.shape[1] # Size of the input vector (n)
output_dim = y.shape[1]  # Size of the output vector (m)

# Create the MLP model
tf_idf_model = mlp_model(input_dim, output_dim)

# Model summary to see the architecture
tf_idf_model.summary()


# X = np.random.rand(number_of_samples, input_dim)
# y = np.random.randint(output_dim, size=(number_of_samples, output_dim))
# y = tf.keras.utils.to_categorical(y, num_classes=output_dim)

# Train the model
tf_idf_model.fit(X, y, epochs=100, batch_size=32)

# Note: In this example, 'X' should be a NumPy array or a TensorFlow tensor with shape (number_of_samples, input_dim),
# where 'number_of_samples' is the total number of input vectors you have, and 'y' should be the corresponding labels
# for each input vector, one-hot encoded to have a shape of (number_of_samples, output_dim).


In [ ]:
tf_idf_decoder_model = pickle.load(open('../ex_difStoryGen/01_model_tfidf.p', 'rb'))

In [ ]:
tf_idf_decoder_model.summary()

100%|██████████| 1402/1402 [00:00<00:00, 63002.64it/s]


In [270]:
convertor_model.predict()

In [271]:
X.shape

(1402, 64)

In [276]:
y.shape

(1402, 128)